# CH5440: Multivariate Data Analysis for Process Modelling

#### Tutorial #3 will cover:

1. Preprocessing data for regression tasks
2. Simple and multiple linear regression
3. Confidence intervals for linear regression estimates (coefficients and predictions)
4. Model evaluation using key metrics and plots
5. Mapping non-linear relations to OLS framework

In [1]:
%pip install scikit-learn statsmodels

Note: you may need to restart the kernel to use updated packages.


Importing Required Libraries:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
import statsmodels.api as sm

Defining function for evaluation of models:

In [3]:
def adjusted_r2(y_true, y_pred,n,p):

    r2 = r2_score(y_true, y_pred)
    adj_r2 = 1 - (1 - r2) * ((n - 1) / (n - p - 1))
    return adj_r2

In [3]:
def metrics(y_test,predictions):
    residuals=y_test-predictions

    print(f'\nThe Mean Absolute Error on the test data is: {mean_absolute_error(y_test,predictions):.3f}\n')
    print(f'\nThe RMSE on the test data is: {mean_squared_error(y_test,predictions)**0.5:.3f}\n')
    print(f'\nThe R2 value is: {r2_score(y_test,predictions):.3f}\n')

    fig,axes = plt.subplots(nrows=1,ncols=3,figsize=(16,6),dpi=150)

    ### scatter plot of y_test and predictions
    axes[0].scatter(y_test, predictions)
    axes[0].set_xlabel('Given Test Values')
    axes[0].set_ylabel('Predicted Values')
    axes[0].set_title('Scatter: Test vs. Predicted Values')
    axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], color='red', linestyle='--')  # Diagonal line

    ### residual plot
    axes[1].scatter(predictions, residuals)
    axes[1].axhline(y=0, color='red', linestyle='--')  # Horizontal line at zero
    axes[1].set_xlabel('Predicted Values')
    axes[1].set_ylabel('Residuals')
    axes[1].set_title('Residuals Plot')

    # Q-Q plot
    sm.qqplot(residuals, line='s',ax=axes[2])  # 's' for standardized line
    axes[2].set_title('Q-Q Plot')

    plt.subplots_adjust(wspace=0.3)
    plt.show()


# Preproceesing Steps

### Cleaning data

Load and check dataset info: We'll be working with Advertising dataset which we already explored in Linear Algebra tutorial.

In [4]:
advert=pd.read_csv('Advertising_unclean.csv')
advert.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         194 non-null    float64
 1   radio      194 non-null    float64
 2   newspaper  193 non-null    float64
 3   sales      200 non-null    float64
dtypes: float64(4)
memory usage: 6.4 KB


There are some missing values in this dataset, so let's see, how we can clean up the dataframe, so it's ready for carrying out regression analysis.

In [ ]:
### displaying all rows which contain any null values
advert[advert.isnull().any(axis=1)]

There are some rows which contain null values for TV, radio and newspaper columns. We can drop those rows, as they have too many missing values.

In [ ]:
advert[advert.drop('sales',axis=1).isnull().all(axis=1)]

In [ ]:
row_to_drop=[14,46,83,109,145,195]
advert = advert.drop(row_to_drop)
advert.reset_index(drop=True, inplace=True)
advert

Checking if any more invalid entries are present:

In [ ]:
advert[advert.isnull().any(axis=1)]

As only value for newspaper column is missing for this row, we can set it equal to the mean of all newspaper entries as an approximation.

In [ ]:
advert.iloc[193,2]=advert['newspaper'].mean()

In [ ]:
advert.isnull().sum()

Dataset is clean now.

### Splitting data into Training and Test sets

Defining X and y for regression:

In [ ]:
X,y=advert.drop('sales',axis=1).values,advert['sales'].values.reshape(-1,1)

Splitting helps in understand how well the model performs on unseen data, in this case, the test data.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### One Hot Encoding

Below is the Fish Market dataset, where we have different characteristics of fish - species and physical measurement. The weight of the fish is the target variable, and we are interested in building a model to predict the weight of a fish given its characteristics.

In [ ]:
fish=pd.read_csv('Fish.csv')
fish.head()

The 'Species' column contains categorical data. We can one-hot encode it to convert to a numerical format, which is necessary for algorithms like linear regression which require numerical input.

In [ ]:
fish = pd.get_dummies(fish, columns=['Species'], drop_first=True)

In [ ]:
fish.head()

# Simple Linear Regression

Below we are synthetically generating data for 'Exam Score' vs 'Hours Studied' to understand how linear regression can be implemented in python.

In [ ]:
np.random.seed(42)
X = np.random.rand(100, 1) * 10  # Feature: Hours studied (0-10)
y = 2.5 * X + np.random.randn(100, 1) * 2  # Target: Exam scores with some noise

# Convert to a DataFrame
example = pd.DataFrame(data={'Hours_Studied': X.flatten(), 'Exam_Score': y.flatten()})
example.head()

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(example[['Hours_Studied']].values.reshape(-1,1), example['Exam_Score'].values.reshape(-1,1), test_size=0.2, random_state=42)

Using OLS formula and matrix operations to find the coefficients of the linear regression model:

In [ ]:
X0=np.ones(X_train.shape[0]).reshape(-1,1)
X_new=np.hstack((X0,X_train.reshape(-1,1)))
beta=np.linalg.inv(X_new.T @ X_new) @ (X_new.T @ y_train)
print(f'The intercept is : {beta[0][0]:.3f} and slope is {beta[1][0]:.3f}')

Using sklearn's inbuilt fucntion for performing linear regression:

In [ ]:
# Create and train the linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

##evaluate

metrics(y_test,predictions)

Understanding evaluation metrics:

MAE and RMSE are in the same unit as the dependent variable 'y', and are a measure of error.
The r2 value indicates how well the model is able to explain the variance in the dependent variable 'y'

1. The test vs predicted values scatter plot helps assess how well the model predictions match the actual test values. Points close to the 45 deg line indicate good predictions.
2. A residual plot shows the difference between actual test values and predicted values. If the residuals are randomly scattered around zero, this indicates that the model captures the underlying pattern well, and the assumptions of linear regression are likely satisfied
3. A Q-Q plot helps assess whether the residuals follow a normal distribution, which is an important assumption in linear regression.

### Comparing results using OLS:

In [ ]:
print(f'The coeffient of the model is {model.coef_[0][0]:.3f} and the intercept obtained is {model.intercept_[0]:.3f}')

In [ ]:
beta

Thus, from above results we can see that sklearn's LinearRegression uses OLS for computation.

# Multiple Linear Regression

We go back to the advertising dataset and explore how to do multiple linear regression. We use statsmodels.api for linear regression this time. This library is useful for detailed statistical analysis and inference from the linear regression model, and in this case we can use it for looking at the confidence intervals for the regression coefficients and predictions.

In [ ]:
X,y=advert.drop('sales',axis=1).values,advert['sales'].values.reshape(-1,1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

X_train=sm.add_constant(X_train)
X_test=sm.add_constant(X_test)

model = sm.OLS(y_train, X_train).fit()

# Step 5: Print the model summary
print(model.summary())

conf_int = model.conf_int()
print("\nConfidence Intervals for Coefficients:\n")
print(conf_int)

predictions = model.get_prediction(X_test)
pred_int = predictions.conf_int()

plt.scatter(y_test, predictions.predicted_mean, label='Predicted vs Actual', alpha=0.5)
plt.plot(y_test, y_test, color='red', linestyle='--', label='Perfect Prediction')
plt.fill_between(y_test.flatten(), pred_int[:, 0], pred_int[:, 1], color='blue', alpha=0.2, label='95% Confidence Interval')
plt.title('Predictions and Confidence Intervals')
plt.xlabel('Actual Values')
plt.ylabel('Predicted Values')
plt.legend()
plt.show()

In [ ]:
metrics(y_test,predictions.predicted_mean.reshape(-1,1))

Understanding adjusted r2 metric:

Let's add an irrelevant feature to the above advertisement data:

In [ ]:
X,y=advert.drop('sales',axis=1).values,advert['sales'].values.reshape(-1,1)
np.random.seed(0)
random_features = np.random.rand(X.shape[0], 10)  # Adding 10 random noise features
X_new = np.hstack((X, random_features))

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.3, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Make predictions
predictions = model.predict(X_test)

##evaluate

metrics(y_test,predictions)

print(f'adjusted r2 is: {adjusted_r2(y_test,predictions,X_test.shape[0],X_test.shape[1]):.3f}')

### Mapping non-linear relations to OLS/linear framework

By transforming non-linear relationships into a linear form, you can apply familiar techniques and interpret the results in a straightforward manner.

Given below is an example, where we have been given enzyme rate measurements and substrate concentration. We want to find a suitable model which describes the kinetics of the system.

In [ ]:
kinetic=pd.read_csv('kinetic_dataset_2.csv')
kinetic.head()

Using linear regression:

In [ ]:
X,y=kinetic['substrate'].values.reshape(-1,1),kinetic['reaction_rate'].values.reshape(-1,1)
X_train,X_test,y_train,y_test=train_test_split(X, y, test_size=0.3, random_state=42)
model=LinearRegression()
model.fit(X_train,y_train)
predictions=model.predict(X_test)
metrics(y_test,predictions)

The linear regression model has a poor r2 value, meaning the model is unable to explain variation in rate measurement data.

Let's try with polynomial regression. But how to select degree of suitable polynomial? We can test polynomials from degree 1 to degree 5 and see which is a better fit model.

In [ ]:
degrees = range(1, 6)
results = []
rmse=[]
mae=[]
r2=[]

X,y=kinetic['substrate'].values.reshape(-1,1),kinetic['reaction_rate'].values.reshape(-1,1)

for degree in degrees:
    poly = PolynomialFeatures(degree=degree)
    X_poly = poly.fit_transform(X)

    X_poly=sm.add_constant(X_poly)
    X_train,X_test,y_train,y_test=train_test_split(X_poly, y, test_size=0.3, random_state=42)
    model = sm.OLS(y_train, X_train).fit()
    predictions = model.get_prediction(X_test)

    coefficients = model.params
    p_values = model.pvalues
    results.append((degree, coefficients, p_values))
    mae.append(mean_absolute_error(y_test,predictions.predicted_mean.reshape(-1,1)))
    rmse.append(mean_squared_error(y_test,predictions.predicted_mean.reshape(-1,1))**0.5)
    r2.append(r2_score(y_test,predictions.predicted_mean.reshape(-1,1)))


# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Degree', 'Coefficients', 'P-values'])

In [ ]:
results_df

From above results on coefficients and p-values, all the p-values are less than 0.05 (signicance level for our case here), so all these polynomials seem to be significant as per this analysis. Let's check the metrics of each of these polynomials to narrow down on the better fitting model.

In [ ]:
fig,ax1 = plt.subplots(figsize=(16,6),dpi=150)


ax1.plot(degrees, mae)
ax1.set_xlabel('Degree of Polynomial')
ax1.set_ylabel('MAE')
ax1.set_title('MAE and R2 score vs Degree Plot')


ax2=ax1.twinx()
ax2.plot(degrees, r2)
ax2.set_xlabel('Degree of Polynomial')
ax2.set_ylabel('R2 value')


plt.subplots_adjust(wspace=0.3)
plt.show()

The elbow method provides a straightforward way to visualize and select the best polynomial degree by analyzing how the model's error changes with increasing complexity. The degree corresponding to this elbow point is considered optimal for the polynomial regression model

The above plot shows us degree 2 polynomial is giving the best performance, so let's implement and check it.

In [ ]:
poly_converter=PolynomialFeatures(degree=2)
X_poly=poly_converter.fit_transform(X)
X_train,X_test,y_train,y_test=train_test_split(X_poly, y, test_size=0.3, random_state=42)
model=LinearRegression()
model.fit(X_train,y_train)
predictions=model.predict(X_test)
metrics(y_test,predictions)

There is an improved r2 value and lower mae and mse observed with polynomial mapping, indicating the variables contain some non-linear relationships.